In [1]:
import xlwings as xw
import re
import pandas as pd

In [2]:
# Excel File 읽기
xw_bom = xw.Book(r'example/bom_fpa.xlsx')
xw_cost = xw.Book(r'example/cost.xlsx')
xw_inventory = xw.Book(r'example/inventory.xlsx')

In [3]:
# 특정 sheet 읽기
sh_bom = xw_bom.sheets(1)
sh_cost = xw_cost.sheets(1)
sh_inventory = xw_inventory.sheets(1)

In [4]:
# Dataframe으로 변환
df_bomlist = sh_bom.range('A1').options(pd.DataFrame, index=False, expand='table').value
df_cost = sh_cost.range('A1').options(pd.DataFrame, index=False, expand='table').value
df_inventory = sh_inventory.range('A1').options(pd.DataFrame, index=False, expand='table').value

In [5]:
#공백제거
df_bomlist['Part No.'] = df_bomlist['Part No.'].str.replace(" ","")
df_cost['품명'] = df_cost['품명'].str.replace(" ","")
df_inventory['자재명'] = df_inventory['자재명'].str.replace(" ","")

In [6]:
# BOM에서 필요한 Column만 추출 및 생성
df_partname = df_bomlist["Part No."].dropna()
df_qty = df_bomlist["Q'TY"].dropna().astype(int)
df_bom = pd.concat([df_partname,df_qty], ignore_index=True, axis=1)
df_bom = df_bom.reset_index(drop=True)
df_bom = df_bom.rename(columns = {0:'품명', 1:'사용수량'})
df_bom['단가'] = [0 for i in range(len(df_bom))]
df_bom['현재고'] = [0 for i in range(len(df_bom))]
df_bom['출고제한수량'] = [0 for i in range(len(df_bom))]
df_bom['프로젝트번호'] = [0 for i in range(len(df_bom))]
# df_partname
# df_qty
# df_bom

In [7]:
# 단가 추가
for k in range(len(df_bom)):
    datafilter = df_cost['품명'].notnull() & df_cost['품명'].str.contains(str(df_bom.iloc[k,0])[:],case=False) # 대소문자 구분X
    if len(df_cost.loc[datafilter,'최종결산월재고단가'].values) == 0:
        df_bom.loc[k,["단가"]] = 0
    elif len(df_cost.loc[datafilter,'최종결산월재고단가'].values) > 1 : # 동일한 부품단가가 있는 경우 마지막값 사용
        df_bom.loc[k,["단가"]] = int(df_cost.loc[datafilter,'최종결산월재고단가'].values[-1])
    else:
        df_bom.loc[k,["단가"]] = int(df_cost.loc[datafilter,'최종결산월재고단가'].values)
# df_bom

In [10]:
# 재고 추가
for k in range(len(df_bom)):
    datafilter = df_inventory['자재명'].notnull() & df_inventory['자재명'].str.contains(str(df_bom.iloc[k,0])[:],case=False) # 대소문자 구분X
    if len(df_inventory[datafilter]['프로젝트번호'].values) == 0:
        pass
    else:
        df_bom.loc[k,['현재고']] = str(df_inventory.loc[datafilter,'현재고'].values)
        df_bom.loc[k,['출고제한수량']] = str(df_inventory.loc[datafilter,'출고제한수량'].values)
        df_bom.loc[k,['프로젝트번호']] = str(df_inventory.loc[datafilter,'프로젝트번호'].values)

[4. 8. 2.]
[7. 2. 3. 4. 6.]
[316. 600. 173.  28. 200. 500. 300. 310. 555.  63.  28.  58. 140.   2.
 139.]
[ 3.  1.  3.  1. 11.]
[ 2. 10.  4.]
[131.]
[2900.    4.   24.]
[ 2. 10.  8.]
[30.]
[48.]
[ 2.  5.  4. 40. 18.  7. 12. 25.]
[5. 2.]


In [9]:
df_bom

,품명,사용수량,단가,현재고,출고제한수량,프로젝트번호
0,C0603C105J4RACAUTO,114,0,0,0,0
1,C0603C104J5RACAUTO,91,0,0,0,0
2,C0603C103J1RACAUTO,87,0,0,0,0
3,T495A106K010AHA2K0,13,0,0,0,0
4,VRF-04-10-50-04-N,2,114757,[4. 8. 2.],[0. 8. 2.],['G-C4P20B' 'A-STP21A' 'G-MSC20B']
5,VSF-05-20-50-04-G,1,99104,[7. 2. 3. 4. 6.],[0. 0. 3. 4. 5.],['N-PIO21A' 'G-C4P20B' 'A-STP21A' 'A-STP21A' '...
6,HIBISCUS,1,0,0,0,0
7,T495A106K010AHA2K0,29,0,0,0,0
8,T495B686K010AHE600,4,0,0,0,0
9,C0805C105J4RACAUTO,1,0,0,0,0


In [ ]:
# CSV 출력
df_bom.to_csv("result.csv",encoding='cp949')